In [3]:
# 패키지 불러오기
# ===== 기본 라이브러리 =====
import numpy as np
import pandas as pd

# ===== 시각화 =====
import matplotlib.pyplot as plt
import seaborn as sns

# ===== 출력 옵션 =====
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None) # 50
pd.set_option("display.max_colwidth", None)  # 컬럼 내용도 잘리지 않게
pd.set_option("display.width", 1000)

# ===== 시각화 스타일 =====
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

In [4]:
# 예약 데이터, 판매일보 데이터 불러오기
sales_df=pd.read_csv("W:/공용 드라이브/01. 매출/00. Raw/Daily/판매일보_20260127.csv",encoding='cp949')
reserve_df=pd.read_excel("W:/공용 드라이브/01. 매출/00. Raw/Daily/예약매출_20260127.xlsx",index_col=0)
stock_df=pd.read_excel("W:/공용 드라이브/02. 재고/01. 재고현황 (일자별)/2026/1월/0127/타점현재고현황_상세정보_20260127.xlsx",index_col=0)

In [5]:
# 매장명에서 매장코드 분리
extracted = stock_df["매장명"].str.extract(r"\[(\d+)\]\s*(.*)")

stock_df["매장코드"] = pd.to_numeric(extracted[0], errors="coerce")
stock_df["매장명"] = extracted[1].fillna(stock_df["매장명"]).str.strip()

# 날짜 컬럼 통일
reserve_df["일자"] = pd.to_datetime(reserve_df["예약일자"])
sales_df["일자"] = pd.to_datetime(sales_df["매출일자"])

In [36]:
# 매장 분류(매장코드 기준)
# 직영점
NSO = [   1585152, 158561, 158541, 158545, 158548,
            158557, 158564, 1585119, 1585126, 1585135,
            1585156, 158510, 158526, 158553, 158554,
            1585151, 158558, 158551, 158560, 1585144,
            158565, 158507, 158535, 158540, 158524,
            158501, 158547, 158552, 158555, 1585124,
            1585125, 1585143, 1585149, 1585159, 158515,
            158503, 158512, 158525, 158550, 158566,
            1585132, 1585133, 1585134, 158542, 158549,
            158527, 1585162, 1585163, 1585164, 1585167,
            1585170, 1585169]

# 파트너점
NSP = [ 1585122, 1585131, 158521, 158562, 158568,
            1585123, 1585142, 1585158, 1585165, 1585166, 1585157]

# 온라인점
ONLINE= [158523, 158563, 158567, 158585]

# 창고, 창고북부천
STOCK=[158595, 158596, 158599]
STOCK_BUK = [158599]

# FOS (나이키일신)
FOS= [158515]

# 이월매장 : 모다
CARRY_OVER_STORE = [158542, 158549, 158527]

In [13]:
sales_df[sales_df['매장코드']==158515].head(2)

,판매점,매장코드,매장명,매출일자,POS번호,영수번호,매출시간,BRAND,BIZ,시즌,상품코드,상품명,매출구분,매출형태,판매가,수량,총매출,판매가할인금액,판매가할인비율,순매출,소비자가,소비자가할인금액,소비자가할인비율,매출유형,교환/반품사유,맴버구분,일자
159,1585,158515,나이키 일신,2026-01-27,1,1,10:32,NIKE,AP,25SP,03-FN3343-072- 080,AS M NK SOLO SWOOSH BB FLC OH,정상,정상,135000,1,135000,67500,50,67500,135000,67500,50.0,일반매출,,,2026-01-27
160,1585,158515,나이키 일신,2026-01-27,1,1,10:32,NIKE,AP,25SP,03-FN3343-072- 075,AS M NK SOLO SWOOSH BB FLC OH,정상,정상,135000,1,135000,67500,50,67500,135000,67500,50.0,일반매출,,,2026-01-27


In [37]:
target_stores = ONLINE + STOCK + FOS + CARRY_OVER_STORE

sales_df[~sales_df['매장코드'].isin(target_stores)]['매장명'].unique()

array(['신세계영등포', '나이키 바이 부평', '신세계죽전', '신세계명동', '온양', '경기시흥', '나이키신세계의정부',
       '스퀘어원인천연수', '동성로타운', '아이즈빌부평', 'TIMES-SQUARE', '신세계충청', '스타일마켓대전',
       '나이키신세계센텀', '신세계김해', '나이키스타일홍대', '충주', '은행동대연', '유성', '탄방로데오',
       '신세계광주패션스트리트', '나이키스타필드하남', '나이키신세계하남', '나이키신세계대구', '당진',
       '신세계고양스타필드RN', '조던월드오브플라이트서울홍대', '나이키 바이 AK 원주', '동해', 'SSG강남스포츠',
       '스타필드위례', '경기광주', '제천', '원주단구동', '나이키 원주타운', '스타필드부천',
       '나이키WELLCOLLECTIVE갤러리아광', '나이키갤러리아천안', '갤러리아진주', '나이키갤러리아타임월드',
       '이벤WCOMGALL광교', '연향', '광주세정', '조던서울', '이천', '스타필드안성', '현대파크원',
       '나이키신세계대전', '안성', '성남신흥', '나이키 바이 건대', '계산', '광장', '나라', '점촌',
       '신포', '나이키모다대전', '나이키스타필드파주'], dtype=object)

In [30]:
aa= ONLINE + FOS + CARRY_OVER_STORE + NSP + NSO
stock_df[~stock_df['매장코드'].isin(aa)]['매장명'].unique()

array(['DCOM신세계닷', '창고기부', '창고이월반품', '창고북부천'], dtype=object)

In [ ]:

# 날짜 컬럼 통일
reserve_df["일자"] = pd.to_datetime(reserve_df["예약일자"])
sales_df["일자"] = pd.to_datetime(sales_df["매출일자"])

sales_with_reserve_strong = sales_df.merge(
    reserve_df[['매장명', '상품코드', '일자', '포스번호', '예약번호', '전화번호', '판매수량']],
    left_on=['매장명', '상품코드', '일자', 'POS번호'],
    right_on=['매장명', '상품코드', '일자', '포스번호'],
    how='outer',
    suffixes=('_sales', '_reserve')
)

sales_with_reserve_strong.head()

,판매점,매장코드,매장명,매출일자,POS번호,영수번호,매출시간,BRAND,BIZ,시즌,상품코드,상품명,매출구분,매출형태,판매가,수량,총매출,판매가할인금액,판매가할인비율,순매출,소비자가,소비자가할인금액,소비자가할인비율,매출유형,교환/반품사유,맴버구분,일자,포스번호,예약번호,전화번호,판매수량
0,1585,158563,CJ몰닷컴대연,2026-01-27,28,2,15:35,NIKE,FW,26SP,02-343938-637- 160,다이나모 프리 (TD),정상,정상,65000,1,65000,0,0,65000,65000,0,0.0,Wholesale매출,****,,2026-01-27,NaN,NaN,NaN,NaN
1,1585,158563,CJ몰닷컴대연,2026-01-27,28,1,13:55,NIKE,FW,25HO,02-394053-100- 235,WMNS NIKE INITIATOR,정상,정상,109000,1,109000,0,0,109000,109000,0,0.0,Wholesale매출,****,,2026-01-27,NaN,NaN,NaN,NaN
2,1585,158563,CJ몰닷컴대연,2026-01-27,28,1,13:55,NIKE,FW,25HO,02-394053-100- 235,WMNS NIKE INITIATOR,정상,정상,109000,1,109000,0,0,109000,109000,0,0.0,Wholesale매출,****,,2026-01-27,NaN,NaN,NaN,NaN
3,1585,158563,CJ몰닷컴대연,2026-01-27,28,1,13:55,NIKE,FW,25HO,02-394053-100- 240,WMNS NIKE INITIATOR,정상,정상,109000,1,109000,0,0,109000,109000,0,0.0,Wholesale매출,****,,2026-01-27,NaN,NaN,NaN,NaN
4,1585,158563,CJ몰닷컴대연,2026-01-27,28,1,13:55,NIKE,FW,25HO,02-394053-100- 240,WMNS NIKE INITIATOR,정상,정상,109000,1,109000,0,0,109000,109000,0,0.0,Wholesale매출,****,,2026-01-27,NaN,NaN,NaN,NaN


In [31]:
sales_with_reserve_strong.isnull().sum()

판매점            0
매장코드           0
매장명            0
매출일자           0
POS번호          0
영수번호           0
매출시간           0
BRAND          0
BIZ            0
시즌             0
상품코드           0
상품명            0
매출구분           0
매출형태           0
판매가            0
수량             0
총매출            0
판매가할인금액        0
판매가할인비율        0
순매출            0
소비자가           0
소비자가할인금액       0
소비자가할인비율       0
매출유형           0
교환/반품사유        0
맴버구분           0
일자             0
포스번호        5596
예약번호        5596
전화번호        5596
판매수량        5596
dtype: int64

In [36]:
sales_with_reserve_strong['매장명'].unique()

array(['CJ몰닷컴대연', 'D컴신세계강남', 'SSG강남스포츠', 'TIMES-SQUARE', 'WCOM신세계몰',
       '갤러리아진주', '경기광주', '경기시흥', '계산', '광장', '광주세정', '나라', '나이키 바이 AK 원주',
       '나이키 바이 건대', '나이키 바이 부평', '나이키 원주타운', '나이키 일신',
       '나이키WELLCOLLECTIVE갤러리아광', '나이키갤러리아천안', '나이키갤러리아타임월드', '나이키모다대전',
       '나이키스타일홍대', '나이키스타필드파주', '나이키스타필드하남', '나이키신세계대구', '나이키신세계대전',
       '나이키신세계센텀', '나이키신세계의정부', '나이키신세계하남', '당진', '동성로타운', '동해',
       '모다아울렛천안', '모다인천북항', '브이플러스연수', '성남신흥', '스퀘어원인천연수', '스타일마켓대전',
       '스타필드부천', '스타필드안성', '스타필드위례', '신세계고양스타필드RN', '신세계광주패션스트리트',
       '신세계김해', '신세계명동', '신세계영등포', '신세계죽전', '신세계충청', '신포', '아이즈빌부평', '안성',
       '연향', '온양', '원주단구동', '유성', '은행동대연', '이벤WCOMGALL광교', '이천', '점촌',
       '제천', '조던서울', '조던월드오브플라이트서울홍대', '충주', '탄방로데오', '현대파크원'],
      dtype=object)

In [33]:
print(len(sales_df)-len(sales_with_reserve_strong))
print(len(reserve_df))

-2
187


In [35]:
# 1. 매칭 키 중복 확인
print("=== 매칭 키 중복 확인 ===")

# sales_df에서 매칭 키 중복 체크
sales_key_cols = ['매장명', '상품코드', '매출일자', 'POS번호']
sales_duplicates = sales_df.groupby(sales_key_cols).size()
sales_duplicates = sales_duplicates[sales_duplicates > 1]

print(f"\nsales_df에서 매칭 키 중복: {sales_duplicates.sum()} 건")
print(f"중복된 키 그룹 수: {len(sales_duplicates)}")

if len(sales_duplicates) > 0:
    print("\n중복 예시:")
    for key, count in sales_duplicates.head(3).items():
        print(f"\n{key}: {count}건")
        mask = (
            (sales_df['매장명'] == key[0]) &
            (sales_df['상품코드'] == key[1]) &
            (sales_df['매출일자'] == key[2]) &
            (sales_df['POS번호'] == key[3])
        )
        print(sales_df[mask][['영수번호', '매장명', '상품코드', '매출일자', 'POS번호', '매출구분']].to_string(index=False))



=== 매칭 키 중복 확인 ===

sales_df에서 매칭 키 중복: 1871 건
중복된 키 그룹 수: 563

중복 예시:

('CJ몰닷컴대연', '02-394053-100-  235', '2026-01-27', 28): 2건
 영수번호     매장명                상품코드       매출일자  POS번호 매출구분
    1 CJ몰닷컴대연 02-394053-100-  235 2026-01-27     28   정상
    1 CJ몰닷컴대연 02-394053-100-  235 2026-01-27     28   정상

('CJ몰닷컴대연', '02-394053-100-  240', '2026-01-27', 28): 6건
 영수번호     매장명                상품코드       매출일자  POS번호 매출구분
    1 CJ몰닷컴대연 02-394053-100-  240 2026-01-27     28   정상
    1 CJ몰닷컴대연 02-394053-100-  240 2026-01-27     28   정상
    1 CJ몰닷컴대연 02-394053-100-  240 2026-01-27     28   정상
    1 CJ몰닷컴대연 02-394053-100-  240 2026-01-27     28   정상
    1 CJ몰닷컴대연 02-394053-100-  240 2026-01-27     28   정상
    1 CJ몰닷컴대연 02-394053-100-  240 2026-01-27     28   정상

('CJ몰닷컴대연', '02-394053-100-  250', '2026-01-27', 28): 4건
 영수번호     매장명                상품코드       매출일자  POS번호 매출구분
    1 CJ몰닷컴대연 02-394053-100-  250 2026-01-27     28   정상
    1 CJ몰닷컴대연 02-394053-100-  250 2026-01-27     28   정상
    1 CJ몰닷컴대연 

In [30]:
sales_with_reserve_strong[(sales_with_reserve_strong['매장명']=='모다인천북항') & (sales_with_reserve_strong['영수번호']==29)]

,판매점,매장코드,매장명,매출일자,POS번호,영수번호,매출시간,BRAND,BIZ,시즌,상품코드,상품명,매출구분,매출형태,판매가,수량,총매출,판매가할인금액,판매가할인비율,순매출,소비자가,소비자가할인금액,소비자가할인비율,매출유형,교환/반품사유,맴버구분,일자,포스번호,예약번호,전화번호,판매수량
4004,1585,158549,모다인천북항,2026-01-27,1,29,16:16,NIKE,FW,26SP,02-FJ7765-120- 235,W NIKE JOURNEY RUN,반품,정상,119000,-1,-119000,-17300,15,-101700,119000,-17300,14.537815,일반매출,고객 변심,,2026-01-27,1.0,2.026013e+13,7701.0,1.0
4007,1585,158549,모다인천북항,2026-01-27,1,29,16:16,NIKE,FW,26SP,02-FJ7765-120- 240,W NIKE JOURNEY RUN,반품,정상,119000,-1,-119000,-17300,15,-101700,119000,-17300,14.537815,일반매출,고객 변심,,2026-01-27,1.0,2.026013e+13,7701.0,1.0
4010,1585,158549,모다인천북항,2026-01-27,1,29,16:16,NIKE,FW,26SP,02-FJ7765-120- 245,W NIKE JOURNEY RUN,반품,정상,119000,-1,-119000,-17300,15,-101700,119000,-17300,14.537815,일반매출,고객 변심,,2026-01-27,1.0,2.026013e+13,7701.0,1.0
4013,1585,158549,모다인천북항,2026-01-27,1,29,16:16,NIKE,FW,26SP,02-FJ7765-120- 250,W NIKE JOURNEY RUN,반품,정상,119000,-1,-119000,-17300,15,-101700,119000,-17300,14.537815,일반매출,고객 변심,,2026-01-27,1.0,2.026013e+13,7701.0,1.0
4024,1585,158549,모다인천북항,2026-01-27,1,29,16:16,NIKE,FW,25HO,02-HQ7002-001- 240,W NIKE VOMERO 18 GTX,반품,정상,209000,-1,-209000,-20900,10,-188100,209000,-20900,10.000000,일반매출,고객 변심,,2026-01-27,1.0,2.026013e+13,7701.0,1.0
4038,1585,158549,모다인천북항,2026-01-27,1,29,16:16,NIKE,AP,26SP,03-FB7549-010- 080,AS M NK DF UNLIMITED PANT TPR,반품,정상,99000,-2,-198000,0,0,-198000,99000,0,0.000000,일반매출,고객 변심,,2026-01-27,1.0,2.026013e+13,7701.0,2.0
4041,1585,158549,모다인천북항,2026-01-27,1,29,16:16,NIKE,AP,26SP,03-FB7549-010- 085,AS M NK DF UNLIMITED PANT TPR,반품,정상,99000,-1,-99000,0,0,-99000,99000,0,0.000000,일반매출,고객 변심,,2026-01-27,1.0,2.026013e+13,7701.0,1.0
4044,1585,158549,모다인천북항,2026-01-27,1,29,16:16,NIKE,AP,26SP,03-FB7549-010- 090,AS M NK DF UNLIMITED PANT TPR,반품,정상,99000,-1,-99000,0,0,-99000,99000,0,0.000000,일반매출,고객 변심,,2026-01-27,1.0,2.026013e+13,7701.0,1.0
4046,1585,158549,모다인천북항,2026-01-27,1,29,16:16,NIKE,AP,26SP,03-FB7549-010- 095,AS M NK DF UNLIMITED PANT TPR,반품,정상,99000,-1,-99000,0,0,-99000,99000,0,0.000000,일반매출,고객 변심,,2026-01-27,1.0,2.026013e+13,7701.0,1.0
4050,1585,158549,모다인천북항,2026-01-27,1,29,16:16,NIKE,AP,26SP,03-FB7552-010- 095,AS M NK RPL UNLIMITED JKT,반품,정상,139000,-2,-278000,0,0,-278000,139000,0,0.000000,일반매출,고객 변심,,2026-01-27,1.0,2.026013e+13,7701.0,2.0


In [ ]:
# ============================================
# 1단계: 예약취소 데이터만 추출
# ============================================
cancel_df = reserve_df[reserve_df['예약처리여부'] == '예약취소'].copy()

print(f"전체 예약취소 건수: {len(cancel_df)}")
print(f"전체 reserve_df 건수: {len(reserve_df)}")

# ============================================
# 2단계: 중복 취소 케이스 식별
# ============================================

# 2-1. 매장명 + 상품코드 + 전화번호로 그룹화
grouping_cols = ['매장명', '상품코드', '전화번호']

# 각 그룹별 취소 건수 계산
cancel_counts = cancel_df.groupby(grouping_cols).size().reset_index(name='취소건수')

# 2번 이상 취소된 케이스만 필터링
duplicate_cancels = cancel_counts[cancel_counts['취소건수'] >= 2]

print(f"\n중복 취소 케이스 수: {len(duplicate_cancels)}")
print(f"총 중복 취소 건수: {duplicate_cancels['취소건수'].sum()}")

# ============================================
# 3단계: 중복 케이스 상세 분석
# ============================================

# 중복 케이스의 상세 정보 추출
duplicate_details = cancel_df.merge(
    duplicate_cancels[grouping_cols],
    on=grouping_cols,
    how='inner'
).sort_values(grouping_cols + ['취소일자'])

print(f"\n중복 취소 상세 레코드 수: {len(duplicate_details)}")

# 중복 케이스를 유형별로 분류
duplicate_details['취소일자_str'] = duplicate_details['취소일자'].astype(str)

# A. 명백한 중복 (모든 주요 컬럼이 동일)
exact_duplicates = duplicate_details.groupby(
    grouping_cols + ['취소일자', '예약번호', '상품명']
).size().reset_index(name='동일건수')
exact_duplicates = exact_duplicates[exact_duplicates['동일건수'] >= 2]

print(f"\n[A] 명백한 중복 (완전히 동일): {len(exact_duplicates)} 그룹")

# B. 다른 날짜에 취소된 케이스
different_date_cancels = duplicate_details.groupby(grouping_cols).agg({
    '취소일자': ['count', 'nunique'],
    '예약번호': 'nunique',
    '상품명': 'first'
}).reset_index()

different_date_cancels.columns = ['매장명', '상품코드', '전화번호',
                                   '취소총건수', '취소일자수', '예약번호수', '상품명']
different_date_cancels = different_date_cancels[
    (different_date_cancels['취소총건수'] >= 2) &
    (different_date_cancels['취소일자수'] >= 2)
]

print(f"[B] 서로 다른 날짜에 취소: {len(different_date_cancels)} 케이스")

# ============================================
# 4단계: sales_df와 교차 검증
# ============================================

# 중복 케이스에 대해 sales_df에서 관련 데이터 찾기
join_cols = ['매장명', '영수번호', '상품코드']

# reserve_df와 sales_df 조인 (중복 케이스만)
duplicate_sales = duplicate_details.merge(
    sales_df,
    left_on=['매장명', '영수증번호', '상품코드'],
    right_on=join_cols,
    how='left',
    suffixes=('_reserve', '_sales')
)

print(f"\n중복 케이스 중 sales_df와 매칭된 건수: {duplicate_sales['영수번호'].notna().sum()}")

# 매칭된 데이터에서 매출구분, 교환/반품사유 확인
if len(duplicate_sales) > 0:
    print("\n[매출구분 분포]")
    print(duplicate_sales['매출구분'].value_counts())

    print("\n[교환/반품사유 분포]")
    print(duplicate_sales['교환/반품사유'].value_counts(dropna=False))

# ============================================
# 5단계: 결과 요약 및 저장
# ============================================

# 케이스별 상세 정보 출력
print("\n" + "="*80)
print("중복 취소 케이스 샘플 (상위 10건)")
print("="*80)

sample_cases = duplicate_details.groupby(grouping_cols).head(10)
display_cols = ['매장명', '전화번호', '상품코드', '상품명', '예약번호',
                '취소일자', '예약일자', '매출구분']
print(sample_cases[display_cols].to_string(index=False))

# ============================================
# 6단계: 추가 분석 - 재예약 가능성 확인
# ============================================

# 같은 고객(전화번호)이 같은 상품을 여러 번 예약했는지 확인
customer_product_history = reserve_df.groupby(['전화번호', '상품코드']).agg({
    '예약일자': 'count',
    '예약번호': 'nunique',
    '예약처리여부': lambda x: x.value_counts().to_dict()
}).reset_index()

customer_product_history.columns = ['전화번호', '상품코드', '총예약건수', '예약번호수', '처리여부분포']
customer_product_history = customer_product_history[customer_product_history['총예약건수'] >= 2]

print(f"\n동일 상품을 2회 이상 예약한 고객 수: {len(customer_product_history)}")

# 중복 취소 케이스 중 재예약 이력이 있는지 확인
reorder_cases = duplicate_details.merge(
    customer_product_history[customer_product_history['총예약건수'] >= 3],
    on=['전화번호', '상품코드'],
    how='inner'
)

print(f"중복 취소 케이스 중 재예약 이력 가능성: {len(reorder_cases)} 건")

전체 예약취소 건수: 17
전체 reserve_df 건수: 187

중복 취소 케이스 수: 0
총 중복 취소 건수: 0

중복 취소 상세 레코드 수: 0

[A] 명백한 중복 (완전히 동일): 0 그룹
[B] 서로 다른 날짜에 취소: 0 케이스

중복 케이스 중 sales_df와 매칭된 건수: 0

중복 취소 케이스 샘플 (상위 10건)
Empty DataFrame
Columns: [매장명, 전화번호, 상품코드, 상품명, 예약번호, 취소일자, 예약일자, 매출구분]
Index: []

동일 상품을 2회 이상 예약한 고객 수: 1
중복 취소 케이스 중 재예약 이력 가능성: 0 건


In [28]:
sales_df['매출유형'].value_counts()

매출유형
Wholesale매출    5554
일반매출           3847
mPOS매출           51
CTM매출             8
Name: count, dtype: int64

In [15]:
sales_df[sales_df['매장명']=='나이키신세계대구'].groupby(['POS번호','영수번호']).size()

POS번호  영수번호
4      1       1
       2       2
       3       1
       4       1
       5       7
              ..
8      34      1
       35      1
       36      2
       37      1
       38      2
Length: 186, dtype: int64

In [23]:
sales_df[(sales_df['매장명']=='나이키신세계대구') & (sales_df['영수번호']==5)]

,판매점,매장코드,매장명,매출일자,POS번호,영수번호,매출시간,BRAND,BIZ,시즌,상품코드,상품명,매출구분,매출형태,판매가,수량,총매출,판매가할인금액,판매가할인비율,순매출,소비자가,소비자가할인금액,소비자가할인비율,매출유형,교환/반품사유,맴버구분
2029,1585,158561,나이키신세계대구,2026-01-26,4,5,11:26,NIKE,AP,26SP,03-FZ5516-100- 080,K NSW RPL WR HD JKT,정상,정상,79000,1,79000,0,0,79000,79000,0,0.0,일반매출,,
2030,1585,158561,나이키신세계대구,2026-01-26,4,5,11:26,NIKE,AP,25FA,03-HF6459-051- 070,AS W NSW NK CHLL FT MR OH SWTP,정상,정상,95000,1,95000,0,0,95000,95000,0,0.0,일반매출,,
2031,1585,158561,나이키신세계대구,2026-01-26,4,5,11:26,NIKE,AP,26SP,03-HQ8185-010- 065,AS W NK 24.7 DF MR JOGGER SOFT,정상,정상,119000,1,119000,0,0,119000,119000,0,0.0,일반매출,,
2032,1585,158561,나이키신세계대구,2026-01-26,4,5,11:26,NIKE,AP,26SP,03-DD4747-211- 100,AS M NK RPL MILER JKT,정상,정상,115000,1,115000,0,0,115000,115000,0,0.0,일반매출,,
2033,1585,158561,나이키신세계대구,2026-01-26,4,5,11:26,NIKE,AP,26SP,03-II0287-010- 080,K NSW TEE AIR,정상,정상,35000,1,35000,0,0,35000,35000,0,0.0,일반매출,,
2034,1585,158561,나이키신세계대구,2026-01-26,4,5,11:26,NIKE,AP,25HO,03-DO7393-101- 100,AS M NSW PREM ESSNTL SUST TEE,정상,정상,59000,1,59000,0,0,59000,59000,0,0.0,일반매출,,
2035,1585,158561,나이키신세계대구,2026-01-26,4,5,11:26,NIKE,AP,26SP,03-HV4549-100- 100,AS M NK UV RPL STRIDE JACKET,정상,정상,155000,1,155000,0,0,155000,155000,0,0.0,일반매출,,
2069,1585,158561,나이키신세계대구,2026-01-26,6,5,11:13,NIKE,AP,26SP,03-IH5086-010- 100,AS DB M NK TEE OS LS BOOKER,정상,정상,65000,1,65000,0,0,65000,65000,0,0.0,일반매출,,
2070,1585,158561,나이키신세계대구,2026-01-26,6,5,11:13,NIKE,AP,26SP,03-IF0722-010- 080,AS M NK CLUB OS FT PANT,정상,정상,105000,1,105000,0,0,105000,105000,0,0.0,일반매출,,
2071,1585,158561,나이키신세계대구,2026-01-26,6,5,11:13,NIKE,VM,22SU,01-G25101- -,Reusable Tote Bag S,정상,정상,2000,1,2000,0,0,2000,2000,0,0.0,일반매출,,


In [ ]:
dup_cancel = (
    reserve_df
    .query("예약처리여부 == '취소'")
    .groupby(["예약번호", "상품코드"])
    .agg(
        취소행수=("라인번호", "count"),
        라인수=("라인번호", "nunique"),
        취소일자수=("취소일자", "nunique")
    )
    .reset_index()
    .query("취소행수 > 1")
)

dup_cancel.head()
